In [ ]:
import tensorflow as tf
from tensorflow.keras.models import sequential 
from tensorflow.keras.layers import Embedding,LSTM, control_dependencies
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequence
import numpy as np
import regex as re